In [7]:
'''READ ME: 

This is a Python Jupyter Notebook designed to convert DGN Surfaces to GIS and run a slope analysis.

Sebastian Brauer - 2024/06/26
'''

import arcpy
from datetime import datetime

arcpy.env.workspace = r'C:\GIS_Scripting\Geodatabases\Slope Rasters.gdb' # CHANGE if using in a different project
arcpy.env.overwriteOutput = True

input_project_name = 'Marshall_TH_19' # REQUIRED USER INPUT always start with an underscore and then come up with a name for the project. The goal is to come up with a unique name, the city and TH make a good combo, feel free to add your last name as well. 
input_county_name = 'Lyon'# REQUIRED USER INPUT in order to get the right projection. Please capitalize the first letter of the county, feel free to review the dictionary if needed.
input_dgn_path = r'C:\Users\sbrauer\Downloads\TopSurface_County.dgn' # REQUIRED USER INPUT in order to find the folder with the data needed for the analysis

# Here is a dictionary that relates every county to a spatial projection number.

mn_county_systems_epsg = {
    "Aitkin": 103700,
    "Anoka": 103708,
    "Becker": 103709,
    "Beltrami North": 103710,
    "Beltrami South": 103711,
    "Benton": 103712,
    "Big Stone": 103713,
    "Blue Earth": 103714,
    "Brown": 103715,
    "Carlton": 103716,
    "Carver": 103717,
    "Cass North": 103718,
    "Cass South": 103719,
    "Chippewa": 103720,
    "Chisago": 103721,
    "Clay": 103701,
    "Clearwater": 103702,
    "Cook North": 103722,
    "Cook South": 103723,
    "Cottonwood": 103724,
    "Crow Wing": 103725,
    "Dakota": 103726,
    "Dodge": 103727,
    "Douglas": 103728,
    "Faribault": 103729,
    "Fillmore": 103730,
    "Freeborn": 103731,
    "Goodhue": 103732,
    "Grant": 103733,
    "Hennepin": 103734,
    "Houston": 1037235,
    "Hubbard": 103703,
    "Isanti": 103736,
    "Itasca North": 103737,
    "Itasca South": 103738,
    "Jackson": 103739,
    "Kanabec": 103740,
    "Kandiyohi": 103741,
    "Kittson": 103742,
    "Koochiching": 103743,
    "Lac qui Parle": 103744,
    "Lake": 103704,
    "Lake of the Woods North": 103745,
    "Lake of the Woods South": 103746,
    "Le Sueur": 103747,
    "Lincoln": 103748,
    "Lyon": 103749,
    "Mahnomen": 103751,
    "Marshall": 103752,
    "Martin": 103753,
    "McLeod": 103750,
    "Meeker": 103754,
    "Mille Lacs": 103705,
    "Morrison": 103755,
    "Mower": 103756,
    "Murray": 103757,
    "Nicollet": 103758,
    "Nobles": 103759,
    "Norman": 103760,
    "Olmsted": 103761,
    "Ottertail": 103762,
    "Pennington": 103763,
    "Pine": 103764,
    "Pipestone": 103765,
    "Polk": 103766,
    "Pope": 103767,
    "Ramsey": 103768,
    "Red Lake": 103769,
    "Redwood": 103770,
    "Renville": 103771,
    "Rice": 103772,
    "Rock": 103773,
    "Roseau": 103774,
    "St. Louis": 103695,
    "St. Louis Central": 103776,
    "St. Louis North": 103775,
    "St. Louis South": 103777,
    "Scott": 103778,
    "Sherburne": 103779,
    "Sibley": 103780,
    "Stearns": 103781,
    "Steele": 103782,
    "Stevens": 103783,
    "Swift": 103784,
    "Todd": 103785,
    "Traverse": 103786,
    "Wabasha": 103787,
    "Wadena": 103788,
    "Waseca": 103789,
    "Washington": 103706,
    "Watonwan": 103790,
    "Wilkin": 103707,
    "Winona": 103791,
    "Wright": 103792,
    "Yellow Medicine": 103793
}


input_multipatch_path = input_dgn_path + '\Multipatch'


print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
epsg_code = mn_county_systems_epsg.get(input_county_name) # Use the input county name to get the WKID/ESPG
spatial_ref = arcpy.SpatialReference(epsg_code) # Create a Spatial Reference item with this projection WKID/ESPG
arcpy.management.DefineProjection(input_multipatch_path, spatial_ref) # Run the 'Define Projection' tool


cell_size = 0.2
cell_assignment_method="MAXIMUM_HEIGHT"
out_raster = f'Slope_Multipatch_to_Raster_Output_{input_project_name}'
arcpy.conversion.MultipatchToRaster(input_multipatch_path, out_raster, cell_size, cell_assignment_method) # Run the 'Multipatch to Raster' tool
print(f"Conversion Complete: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


out_slope_raster = f'Slope_Raster_{input_project_name}'
output_measurement = "PERCENT_RISE"
arcpy.ddd.Slope(out_raster, out_slope_raster, output_measurement) # Run the 'Slope' tool
print(f"Slope Calculation Complete: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


reclass_field = "Value" # Field that is getting reclassed, will always just be value unless it's a multiband raster
remap = "0 1.999000 1;1.999000 4.999000 2;4.999000 8.299000 3;8.299000 15 4;15 25 5;25 50 6" # Reclass values
out_classify_raster = f'Slope_Reclassify_Output_{input_project_name}'
missing_values = "NODATA" # Leave input NODATA values and NODATA
arcpy.ddd.Reclassify(out_slope_raster, reclass_field, remap, out_classify_raster, missing_values) # Run the 'Reclassify' tool
print(f"ReClass Complete: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


out_polygon_features = f'Slope_Raster_Polygons_{input_project_name}' # Tool parameter
simplify = 'NO_SIMPLIFY' # Tool parameter
raster_field = 'Value' # Tool parameter
create_multipart_features = 'MULTIPLE_OUTER_PART' # Tool parameter
arcpy.conversion.RasterToPolygon(out_classify_raster, out_polygon_features, simplify, raster_field, create_multipart_features) # Run the 'Raster to Polygon' tool
print(f"Script Complete: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Started: 2024-07-03 12:13:50
Conversion Complete: 2024-07-03 12:14:05
Slope Calculation Complete: 2024-07-03 12:14:23
ReClass Complete: 2024-07-03 12:14:37
Raster Conversion Complete: 2024-07-03 12:14:44
